In [1]:
import numpy as np
import os.path as op
from pprint import pformat
import argparse
# EEG utilities
import mne
from mne.preprocessing import ICA, create_eog_epochs
from pyprep.prep_pipeline import PrepPipeline
# BIDS utilities
from mne_bids import BIDSPath, read_raw_bids
from util.io.bids import DataSink

import mne.channels
from mne.io.meas_info import *
from util.io.preprocessing import *

In [58]:
# constants
BIDS_ROOT = '../data/bids'
DERIV_ROOT = op.join(BIDS_ROOT, 'derivatives')
FFR_PASSBAND = (30., 300.)
MICROSTATE_PASSBAND = (1., 30.)
TASK = 'pitch'
sub = '3'
run = '1'

'''
Parameters
----------
sub : str
    Subject ID as in BIDS dataset
'''
# load data
bids_path = BIDSPath(
    root = BIDS_ROOT,
    subject = sub,
    task = TASK,
    run = run,
    datatype = 'eeg'
    )
print(bids_path)
raw = read_raw_bids(bids_path, verbose = False)
events, event_ids = mne.events_from_annotations(raw)

# re-reference eye electrodes to become bipolar EOG
raw.load_data()
def reref(dat):
    dat[0,:] = (dat[1,:] - dat[0,:])
    return dat
raw = raw.apply_function(
    reref,
    picks = ['leog', 'Fp2'],
    channel_wise = False
)
raw = raw.apply_function(
    reref,
    picks = ['reog', 'Fp1'],
    channel_wise = False
)

raw = raw.set_channel_types({'leog': 'eog', 'reog': 'eog'})



../data/bids/sub-3/eeg/sub-3_task-pitch_run-1_eeg.vhdr
Used Annotations descriptions: ['100', '150', '200', '250', '50']
Reading 0 ... 5026249  =      0.000 ...  1005.250 secs...


/tmp/ipykernel_19569/675024540.py:25: RuntimeWarning: The unit for channel(s) Aux1 has changed from NA to V.
  raw = read_raw_bids(bids_path, verbose = False)
/tmp/ipykernel_19569/675024540.py:25: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['leog', 'reog', 'Aux1']
  raw = read_raw_bids(bids_path, verbose = False)
/tmp/ipykernel_19569/675024540.py:25: RuntimeWarning: Not setting position of 1 stim channel found in montage:
['Aux1']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = read_raw_bids(bids_path, verbose = False)


In [3]:
# Does raw have EOG channels now? Yes
raw

<RawBrainVision | sub-3_task-pitch_run-1_eeg.eeg, 65 x 5026250 (1005.2 s), ~2.43 GB, data loaded>

In [59]:
# run PREP pipeline (notch, exclude bad chans, and re-reference)
raw, events = raw.resample(int(2*FFR_PASSBAND[1]), events = events)
np.random.seed(int(sub))
lf = raw.info['line_freq']
prep_params = {
    "ref_chs": "eeg",
    "reref_chs": "eeg",
    "line_freqs": np.arange(lf, FFR_PASSBAND[1], lf)
}
prep = PrepPipeline(
    raw,
    prep_params,
    raw.get_montage(),
    ransac = False,
    random_state = int(sub)
    )

In [ ]:
prep = prep.fit()

In [ ]:
# Extract data from PREP
print('----------------- Extract data from PREP ------------------')
prep_eeg = prep.raw_eeg # get EEG channels from PREP
prep_non_eeg = prep.raw_non_eeg # get non-EEG channels from PREP
raw_data = np.concatenate((prep_eeg.get_data(), prep_non_eeg.get_data())) # combine data from the two

In [68]:
# Create info object for post-PREP data
raw_copy = raw.copy()
print('Create info object for post-PREP data')
new_ch_names = prep_eeg.info['ch_names'] + prep_non_eeg.info['ch_names']
raw_copy = raw_copy.reorder_channels(new_ch_names) # modify the channel names on the original raw data
print(raw_copy.info)
print(raw_copy.info['ch_names'])
print(raw_copy.get_channel_types())

Create info object for post-PREP data
<Info | 12 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 62 EEG, 2 EOG, 1 Stimulus
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 67 items (3 Cardinal, 64 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 60.0
 lowpass: 300.0 Hz
 meas_date: 1924-12-31 15:07:11 UTC
 nchan: 65
 projs: []
 sfreq: 600.0 Hz
 subject_info: 4 items (dict)
>
['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF4', 'F2', 'Iz', 'Cz', 'leog', 'reog', 'Aux1']
['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg',

In [66]:
# # Create new channel types mapping
# new_ch_types = ['eeg']*62 + ['eog']*2 + ['stim']
# print(new_ch_types)
# # mapping = {}
# # for key in new_ch_names:
# #     for value in new_ch_types:
# #         mapping[key] = value
# # raw_copy = raw_copy.set_channel_types(mapping) # modify the channel mappings on the original raw data
# # raw_copy.info['bads'] = [] # already interpolated by PREP
# # raw_info = raw_copy.info # use the modified info from the original raw data object
# # print(raw_info)
# # print(raw_info.get_channel_types())

['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eog', 'eog', 'stim']


In [57]:


# Create info object for post-PREP data
print('Create info object for post-PREP data')
new_ch_names = prep_eeg.info['ch_names'] + prep_non_eeg.info['ch_names']
raw = raw.reorder_channels(new_ch_names) # modify the channel names on the original raw data
print(raw.info)
new_ch_types = ['eeg']*62 + ['eog']*2 + ['stim']
mapping = {}
for key in new_ch_names:
    for value in new_ch_types:
        mapping[key] = value
raw = raw.set_channel_types(mapping) # modify the channel mappings on the original raw data
raw.info['bads'] = [] # already interpolated by PREP
raw_info = raw.info # use the modified info from the original raw data object
print(raw_info)

# Create new raw object
print('Create new raw object')
raw = mne.io.RawArray(raw_data, raw_info) # replace original raw object

----------------- Extract data from PREP ------------------
Create info object for post-PREP data
<Info | 12 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 65 Stimulus
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 67 items (3 Cardinal, 64 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 60.0
 lowpass: 300.0 Hz
 meas_date: 1924-12-31 15:07:11 UTC
 nchan: 65
 projs: []
 sfreq: 600.0 Hz
 subject_info: 4 items (dict)
>
<Info | 12 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 65 Stimulus
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 67 items (3 Cardinal, 64 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 60.0
 lowpass: 300.0 Hz
 meas_date: 1924-12-31 15:07:11 UTC
 nchan: 65
 projs: []
 sfreq: 600.0 Hz
 subject_i

In [51]:
raw_for_ffr = raw.copy().pick(['Cz', 'TP9', 'TP10'])

In [55]:
raw.get_channel_types()

['stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim',
 'stim']

In [ ]:
raw_prep = prep.raw_eeg # replace raw with cleaned version
raw_non_eeg = prep.raw_non_eeg # return the eog
raw_prep = raw_prep.add_channels([raw_non_eeg], force_update_info=True) # combine eeg and non eeg, okay, so this part isn't working
bads = prep.noisy_channels_original
raw_prep.info['bads'] = [] # already interpolated by PREP
raw_copy = raw.copy()
new_ch_names = prep.raw_eeg.info['ch_names'] + raw_non_eeg.info['ch_names']
new_ch_types = ['eeg']*62 + ['eog']*2 + ['stim']
raw_copy = raw_copy.reorder_channels(new_ch_names)
mapping = {}
for key in new_ch_names:
    for value in new_ch_types:
        mapping[key] = value
raw_copy = raw_copy.set_channel_types(mapping)

In [47]:
# need to reorder channels
raw_copy = raw.copy()
new_ch_names = prep.raw_eeg.info['ch_names'] + raw_non_eeg.info['ch_names']
new_ch_types = ['eeg']*62 + ['eog']*2 + ['stim']
raw_copy = raw_copy.reorder_channels(new_ch_names)
mapping = {}
for key in new_ch_names:
    for value in new_ch_types:
        mapping[key] = value
raw_copy = raw_copy.set_channel_types(mapping)

In [48]:
raw.info

<Info | 12 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 62 EEG, 2 EOG, 1 Stimulus
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 67 items (3 Cardinal, 64 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 60.0
 lowpass: 300.0 Hz
 meas_date: 1924-12-31 15:07:11 UTC
 nchan: 65
 projs: []
 sfreq: 600.0 Hz
 subject_info: 4 items (dict)
>

In [39]:
prep_eeg = prep.raw_eeg # get EEG channels from PREP
prep_non_eeg = prep.raw_non_eeg # get non-EEG channels from PREP
raw_data = np.concatenate((prep_eeg.get_data(), prep_non_eeg.get_data())) # combine the two
raw_info = raw.info # copy info from original raw data object
new_ch_names = prep_eeg.info['ch_names'] + raw_non_eeg.info['ch_names']
raw = mne.io.RawArray(raw_data, raw_info) # replace original raw object
raw.info['bads'] = [] # already interpolated by PREP
# raw.info['ch_names'] = new_ch_names
raw

Creating RawArray with float64 data, n_channels=65, n_times=603150
    Range : 0 ... 603149 =      0.000 ...  1005.248 secs
Ready.


RuntimeError: ch_names cannot be set directly. Please use methods inst.add_channels(), inst.drop_channels(), inst.pick_channels(), inst.rename_channels(), inst.reorder_channels() and inst.set_channel_types() instead.

In [32]:
raw.info['ch_types']

KeyError: 'ch_types'

In [29]:
new_ch_names = prep.raw_eeg.info['ch_names'] + raw_non_eeg.info['ch_names']
new_ch_names

['Fp1',
 'Fz',
 'F3',
 'F7',
 'FT9',
 'FC5',
 'FC1',
 'C3',
 'T7',
 'TP9',
 'CP5',
 'CP1',
 'Pz',
 'P3',
 'P7',
 'O1',
 'Oz',
 'O2',
 'P4',
 'P8',
 'TP10',
 'CP6',
 'CP2',
 'C4',
 'T8',
 'FT10',
 'FC6',
 'FC2',
 'F4',
 'F8',
 'Fp2',
 'AF3',
 'AFz',
 'F1',
 'F5',
 'FT7',
 'FC3',
 'C1',
 'C5',
 'TP7',
 'CP3',
 'P1',
 'P5',
 'PO7',
 'PO3',
 'POz',
 'PO4',
 'PO8',
 'P6',
 'P2',
 'CPz',
 'CP4',
 'TP8',
 'C6',
 'C2',
 'FC4',
 'FT8',
 'F6',
 'AF4',
 'F2',
 'Iz',
 'Cz',
 'leog',
 'reog',
 'Aux1']

In [7]:
def add_channels(self, add_list, force_update_info=False):
        """Append new channels to the instance.

        Parameters
        ----------
        add_list : list
            A list of objects to append to self. Must contain all the same
            type as the current object.
        force_update_info : bool
            If True, force the info for objects to be appended to match the
            values in ``self``. This should generally only be used when adding
            stim channels for which important metadata won't be overwritten.

            .. versionadded:: 0.12

        Returns
        -------
        inst : instance of Raw, Epochs, or Evoked
            The modified instance.

        See Also
        --------
        drop_channels

        Notes
        -----
        If ``self`` is a Raw instance that has been preloaded into a
        :obj:`numpy.memmap` instance, the memmap will be resized.
        """
        # avoid circular imports
#         from ..io import BaseRaw, _merge_info
#         from ..epochs import BaseEpochs

#         _validate_type(add_list, (list, tuple), "Input")

        # Object-specific checks
#         for inst in add_list + [self]:
#             print("skip")
# #             _check_preload(inst, "adding channels")
#         if isinstance(self, BaseRaw):
#             con_axis = 0
#             comp_class = BaseRaw
#         elif isinstance(self, BaseEpochs):
#             con_axis = 1
#             comp_class = BaseEpochs
#         else:
#             con_axis = 0
#             comp_class = type(self)
#         for inst in add_list:
#             _validate_type(inst, comp_class, "All input")
        con_axis = 0
        data = [inst._data for inst in [self] + add_list]
#         print("check")
#         print(data)

        # Make sure that all dimensions other than channel axis are the same
        compare_axes = [i for i in range(data[0].ndim) if i != con_axis]
        shapes = np.array([dat.shape for dat in data])[:, compare_axes]
        for shape in shapes:
            if not ((shapes[0] - shape) == 0).all():
                raise ValueError(
                    "All data dimensions except channels must match, got "
                    f"{shapes[0]} != {shape}"
                )
        del shapes

        # Create final data / info objects
        infos = [self.info] + [inst.info for inst in add_list]
        new_info = _merge_info(infos, force_update_to_first=force_update_info)
        
#         print(infos)
#         print(new_info)

        # Now update the attributes
        if (
            isinstance(self._data, np.memmap)
            and con_axis == 0
            and sys.platform != "darwin"
        ):  # resizing not available--no mremap
            # Use a resize and fill in other ones
            out_shape = (sum(d.shape[0] for d in data),) + data[0].shape[1:]
            n_bytes = np.prod(out_shape) * self._data.dtype.itemsize
            self._data.flush()
            self._data.base.resize(n_bytes)
            self._data = np.memmap(
                self._data.filename, mode="r+", dtype=self._data.dtype, shape=out_shape
            )
            assert self._data.shape == out_shape
            assert self._data.nbytes == n_bytes
            offset = len(data[0])
            for d in data[1:]:
                this_len = len(d)
                self._data[offset : offset + this_len] = d
                offset += this_len
        else:
            self._data = np.concatenate(data, axis=con_axis)
        self.info = new_info
        if True:
#         if isinstance(self, BaseRaw):
            self._cals = np.concatenate(
                [getattr(inst, "_cals") for inst in [self] + add_list]
            )
            # We should never use these since data are preloaded, let's just
            # set it to something large and likely to break (2 ** 31 - 1)
            extra_idx = [2147483647] * sum(info["nchan"] for info in infos[1:])
#             print(f'len(r): {len(r)}')
            print(self._read_picks)
            print(f'infos[0]["nchan"]: {infos[0]["nchan"]}')
#             assert all(len(r) == infos[0]["nchan"] for r in self._read_picks)
            self._read_picks = [
                np.concatenate([r, extra_idx]) for r in self._read_picks
            ]
#             assert all(len(r) == self.info["nchan"] for r in self._read_picks)
            for other in add_list:
                self._orig_units.update(other._orig_units)
        elif isinstance(self, BaseEpochs):
            self.picks = np.arange(self._data.shape[1])
            if hasattr(self, "_projector"):
                activate = False if self._do_delayed_proj else self.proj
                self._projector, self.info = setup_proj(
                    self.info, False, activate=activate
                )

        return self

In [17]:
raw_manual_data = np.concatenate((raw_prep_2.get_data(), raw_non_eeg.get_data()))
raw_manual_info = raw.info

In [19]:
raw_manual = mne.io.RawArray(raw_manual_data, raw_manual_info)
raw_manual

Creating RawArray with float64 data, n_channels=65, n_times=603150
    Range : 0 ... 603149 =      0.000 ...  1005.248 secs
Ready.


<RawArray | 65 x 603150 (1005.2 s), ~299.2 MB, data loaded>

In [24]:
# Does raw have EOG channels now? Yes
raw

<RawBrainVision | sub-3_task-pitch_run-1_eeg.eeg, 65 x 603150 (1005.2 s), ~299.2 MB, data loaded>

In [25]:
# Does raw_prep have EOG channels now? No!
raw_prep

<RawBrainVision | sub-3_task-pitch_run-1_eeg.eeg, 62 x 603150 (1005.2 s), ~299.2 MB, data loaded>

In [3]:
## reconstruct the typical FFR/ABR montage we know and love
print('## reconstruct the typical FFR/ABR montage we know and love')
raw_for_ffr = raw.copy().pick(['Cz', 'TP9', 'TP10']) # Cz and mastoids
raw_for_ffr.set_eeg_reference(ref_channels = ['TP9', 'TP10'])
raw_for_ffr = raw_for_ffr.pick(['Cz'])
# and filter (just the highpass, since lowpass applied when downsampling)
raw_for_ffr = raw_for_ffr.filter(l_freq = FFR_PASSBAND[0], h_freq = None)
# then epoch
epochs = mne.Epochs(
    raw_for_ffr,
    events,
    tmin = -.25,
    tmax = .25,
    event_id = event_ids,
    baseline = (-.25, 0.),
    preload = True
)

# drop bad epochs
print('# drop bad epochs')
epochs.drop_bad(reject = dict(eeg = 35e-6))
# # then save epochs for later
# sink = DataSink(DERIV_ROOT, 'preprocess_ffr')
# ffr_fpath = sink.get_path(
#     subject = sub,
#     task = TASK,
#     run = run,
#     desc = 'forFFR',
#     suffix = 'epo',
#     extension = 'fif.gz'
# )
# epochs.save(ffr_fpath, overwrite = True)

## reconstruct the typical FFR/ABR montage we know and love
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Filter length: 265 samples (0.442 s)

Not setting metadata
2400 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 2400 events and 301 original time points ...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


0 bad epochs dropped
# drop bad epochs
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  epoch based on EEG : ['Cz']
    Rejecting  

Number of events,2368
Events,100: 443150: 495200: 472250: 48650: 472
Time range,-0.250 – 0.250 s
Baseline,-0.250 – 0.000 s


In [ ]:
# Does raw have EOG channels now?
raw

# Does raw_for_ffr have EOG channels now?
raw_for_ffr

# Does epochs have EOG channels now?
raw

In [7]:
## now prepare non-epoched data for microstate jazz
# identify bad ICs on weakly highpassed data
print('now prepare non-epoched data for microstate jazz')
print('# identify bad ICs on weakly highpassed data')
raw_for_ica = raw.copy().filter(l_freq = 1., h_freq = None)
epochs_for_ica = mne.Epochs(
    raw_for_ica,
    epochs.events, # same events as FFR epochs
    picks = ['eeg', 'eog'],
    tmin = -.25,
    tmax = .0, # only prestim
    event_id = event_ids,
    baseline = None,
    preload = True
)


now prepare non-epoched data for microstate jazz
# identify bad ICs on weakly highpassed data
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1981 samples (3.302 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
2368 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2368 events and 151 original time points ...


[Parallel(n_jobs=1)]: Done  62 out of  62 | elapsed:    1.9s finished


0 bad epochs dropped


In [11]:
# Check properties
raw_for_ffr

<RawBrainVision | sub-3_task-pitch_run-1_eeg.eeg, 1 x 603150 (1005.2 s), ~4.6 MB, data loaded>

In [ ]:
ica = ICA(n_components = 15, random_state = 0)
ica.fit(epochs_for_ica, picks = ['eeg', 'eog'])

In [ ]:
eog_indices, eog_scores = ica.find_bads_eog(epochs_for_ica, threshold = 1.96)
ica.exclude = eog_indices
# filter to desired bandwidth and remove bad ICs
raw = raw.filter(*MICROSTATE_PASSBAND)
epochs_for_micro = mne.Epochs(
    raw,
    epochs.events, # same events as FFR epochs
    tmin = -.25,
    tmax = .0, # only prestim
    event_id = event_ids,
    baseline = None,
    preload = True
)
ica.apply(epochs_for_micro) # transforms in place
# now we no longer need EOG channels
epochs_for_micro = epochs_for_micro.drop_channels('leog')
epochs_for_micro = epochs_for_micro.drop_channels('reog')
# and save
micro_fpath = sink.get_path(
    subject = sub,
    task = TASK,
    run = run,
    desc = 'forMicrostate',
    suffix = 'epo',
    extension = 'fif.gz'
)
epochs_for_micro.save(micro_fpath, overwrite = True)

# generate a report
report = mne.Report(verbose = True)
report.parse_folder(op.dirname(ffr_fpath), pattern = '*epo.fif.gz', render_bem = False)
if ica.exclude:
    fig_ica_removed = ica.plot_components(ica.exclude, show = False)
    report.add_figure(
        fig_ica_removed,
        title = 'Removed ICA Components',
        section = 'ICA'
    )
bads = prep.noisy_channels_original
html_lines = []
for line in pformat(bads).splitlines():
    html_lines.append('<br/>%s' % line)
html = '\n'.join(html_lines)
report.add_html(html, title = 'Interpolated Channels', section = 'channels')
report.add_html(epochs.info._repr_html_(), title = 'Epochs Info (FFR)', section = 'info')
report.add_html(epochs_for_micro.info._repr_html_(), title = 'Epochs Info (Microstates)', section = 'info')
report.save(op.join(sink.deriv_root, 'sub-%s.html'%sub), overwrite = True)


In [ ]:
raw